In [2]:
from utils.protein_logic import prepare_protein
from utils.ligand_logic import prepare_ligands
import yaml

with open("config.yaml") as f:
    cfg = yaml.safe_load(f)

# Protein
prepare_protein(
    cfg["protein"]["input_pdb"],
    cfg["protein"]["output_pdb"],
    cfg["protein"]["pH"]
)

# Ligands
n = prepare_ligands(
    cfg["ligands"]["input_excel"],
    cfg["ligands"]["smiles_column"],
    cfg["ligands"]["id_column"],
    cfg["ligands"]["output_sdf"],
    cfg["embedding"]["random_seed"]
)

print(f"{n} ligands successfully prepared.")


Prepared 5 ligands
Failed   0 ligands
5 ligands successfully prepared.
